<a href="https://colab.research.google.com/github/mathluva/bert-as-embedder/blob/main/Bert_as_embedder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
#use ! for terminal commands
!pip install bert-for-tf2 #tensorflow2 light version
!pip install sentencepiece #required for BERT-tf2

     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=09f18e52ef4cf6e08f5e0b1adbd782a7133b079b87121d83a0a47ba56f7e3c69
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=33245b4441e28b3e3bb68db1a9269b86e2ac9fafa64dd4811ab940dbe26ed30f
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=257ace655b21c4b4dc644cde828b3fc121e8e96119686915ae563e411c0156f4
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 23.3MB/s 


In [3]:
try:
    %tensorflow_version 2.x #only available in Google colab
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub #used to import the weights from BERT

from tensorflow.keras import layers
import bert #installed in previous step

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x #only available in Google colab`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [4]:
#load files, data preprocessing
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
#label columns
#latin1 is common for western languages
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/trainingandtestdata.zip (Unzipped Files)/training.1600000.processed.noemoticon.csv", 
    header = None,
    names = cols,
    engine = "python",
    encoding = "latin1")

In [6]:
#axis1 column data
#without inplace=True, it would be required to write data = data.drop("...")
data.drop(["id", "date","query", "user"], axis = 1, inplace = True)

In [7]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
#cleaning
#r is regEX (regexr.com for more documentation)
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text() #tweets are not usuable as standard string, need BS to extract string
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ',tweet)#anything behind @symbol with empty space, apply to tweet
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)#? means the s can be there or not
    tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet) #keep only standard characters
    tweet = re.sub(r" +", ' ', tweet) #replace multiple sequences of white space with only one white space
    return tweet

In [9]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [10]:
#process sentiment
data_labels = data.sentiment.values
data_labels[data_labels ==4] =1 #data is using 0 and 4, replace 4 with standard 1

In [16]:
#create BERT layer to have access to metadata for the tokenizer(like vocab size).
#call BERT as a layer, hub is where all pretrained models are located
#trainable = False bc we are not fine-tuning the weights
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable = False) 
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() #way to have acces to vocab
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

We only use the first sentence for BERT inputs so we add the CLS token at the beginning and the SEP token at the end of each sentence.

In [22]:
#the first layer is a BERT layer so
#make inputs suitable for BERT
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]


In [23]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

Dataset Creation

We need to create 3 different inputs for each sentence:
1.  The tokenize version of the sentence created in data_inputs
2. Mask layer: 1 for standard tokens and 0 for padding
3. Segment input: sequence of 0 and 1's , 0  correspond to first sentence and 1 for second sentence 

In [143]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)


#padding mask, compare each element of token with the string "[PAD]"
#not_equal will give us a 1 when not using [PAD] token
def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)



def get_segments(tokens):
    is_sep = np.char.equal(tokens, "[SEP]")
    is_first_sent = np.cumsum(is_sep[::-1])[::-1]
    return 1 - is_first_sent


#determine if 1st or 2nd sentence
#first sentence is 0 "current_id", after [SEP] turn 0 to 1
def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok =="[SEP]":
            current_seg_id = 1- current_seg_id  #once we get to a second [SEP] it will turn back to 0, for sentence pair options in BERT
        return seg_ids

In [144]:
data_inputs[50]

['[CLS]',
 'broadband',
 'plan',
 'a',
 'massive',
 'broken',
 'promise',
 'via',
 'www',
 '.',
 'di',
 '##igo',
 '.',
 'com',
 'taut',
 '##ao',
 'still',
 'waiting',
 'for',
 'broadband',
 'we',
 'are',
 '[SEP]']

In [145]:
get_mask(data_inputs[50])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [146]:
get_segments(data_inputs[50])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [147]:
get_ids(data_inputs[50])

[101,
 19595,
 2933,
 1037,
 5294,
 3714,
 4872,
 3081,
 7479,
 1012,
 4487,
 14031,
 1012,
 4012,
 21642,
 7113,
 2145,
 3403,
 2005,
 19595,
 2057,
 2024,
 102]

We will create padded batches (so we pad sentences for each batch independently), this way we add the minimum of padding tokens possible.  For that, we sort sentences by length, apply padded_batches and then shuffle.

In [148]:
data_with_len = [[sent, data_labels[i] ,len(sent)] 
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len) 
data_with_len.sort(key=lambda x: x[2]) 
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]), 
                get_segments(sent_lab[0])], sent_lab[1])
                    for sent_lab in data_with_len if sent_lab[2]>7]

In [149]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                output_types =(tf.int32, tf.int32))


In [151]:
BATCH_SIZE =32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None,  ), ()))

In [152]:
my_sent = ["[CLS]"] +tokenizer.tokenize("Roses are red.") + ["[SEP]"]
bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0)])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.2793572e-01, -4.1033605e-01, -9.6575540e-01,  9.0731835e-01,
          8.1291467e-01, -1.7417499e-01,  9.1123497e-01,  3.4195280e-01,
         -8.7452203e-01, -9.9998933e-01, -7.7841067e-01,  9.6938545e-01,
          9.8616040e-01,  6.3696516e-01,  9.4863141e-01, -7.5119400e-01,
         -4.5834044e-01, -7.0810521e-01,  4.6209922e-01, -6.5792799e-01,
          7.6041526e-01,  9.9999481e-01, -3.9686257e-01,  3.4416667e-01,
          6.1648947e-01,  9.9440002e-01, -7.7663440e-01,  9.3831664e-01,
          9.5945227e-01,  7.3287970e-01, -6.9343781e-01,  2.9308090e-01,
         -9.9378556e-01, -1.6455242e-01, -9.6701986e-01, -9.9554962e-01,
          5.3293628e-01, -6.8806112e-01,  1.3470631e-02,  2.9818902e-02,
         -9.1835672e-01,  4.2052680e-01,  9.9998897e-01,  2.5267771e-01,
          6.0623682e-01, -3.5075051e-01, -9.9999976e-01,  4.9758595e-01,
         -8.9518762e-01,  9.6256125e-01,  9.4373119e-01,  9.0328616e-01,


In [168]:
class DCNNBERTEmbedding(tf.keras.Model):

    def __init__(self, 
                 nb_filters = 50,
                 FFN_units = 512,
                 nb_classes = 2, 
                 dropout_rate = 0.1,
                 name = 'dcnn'):
        
        super(DCNNBERTEmbedding, self).__init__(name = name)

        #creating layers used in the model, embedded
        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable = False)

        self.bigram = layers.Conv1D(filters = nb_filters, 
                                               kernel_size =2, 
                                                padding = 'valid', 
                                                activation = 'relu')
        
        self.trigram = layers.Conv1D(filters = nb_filters, 
                                               kernel_size =3, 
                                                padding = 'valid', 
                                                activation = 'relu')

        self.fourgram = layers.Conv1D(filters = nb_filters, 
                                               kernel_size =4, 
                                                padding = 'valid', 
                                                activation = 'relu')
        
        self.pool = layers.GlobalMaxPooling1D()

        self.dense_1= layers.Dense(units = FFN_units,
                                    activation = 'relu')
        
        self.dropout = layers.Dropout(rate = dropout_rate)
        if nb_classes:
            self.last_dense = layers.Dense(units = 1, activation = 'sigmoid')

        else: 
            self.last_dense = layers.Dense(units = nb_classes, activation = 'softmax')
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.bigram(x)
        x_2 = self.pool(x_2) 
        x_3 = self.bigram(x)
        x_3 = self.pool(x_3)#(batch_size, nb_filters)

        merged = tf.concat([x_1, x_2, x_3], axis =-1)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

In [169]:
#Training
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [170]:
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES//10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [171]:
Dcnn = DCNNBERTEmbedding(nb_filters = NB_FILTERS,
                         FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)

In [172]:
if NB_CLASSES ==2:
    Dcnn.compile(loss = 'binary_crossentropy',
                    optimizer = 'adam',
                 metrics = ['accuracy'])

else:
    Dcnn.compile(loss = "sparse_categorical_crossentropy",
                        optimizer = 'adam',
                  metrics = ['sparse_categorical_crossentropy'])

In [173]:
checkpoint_path = "/content/drive/MyDrive/Bert-as-embedding"

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep =1)

if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print('Latest checkpoint has been restored')

In [174]:
#custom callback, execute during training
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs = None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [175]:
Dcnn.fit(train_dataset, 
         epochs = NB_EPOCHS,
         callbacks = [MyCustomCallback()])

Epoch 1/5


ValueError: ignored